In [11]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, Dataset
from torch.autograd import Variable
import math
import tqdm
import pdb
from scipy.sparse import csr_matrix, linalg
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import warnings

warnings.filterwarnings(action='ignore')


In [19]:
from PIL import Image
from sklearn.model_selection import train_test_split
import torchvision.transforms as transforms
import torch
from torch.utils.data import DataLoader, Dataset
from torch.autograd import Variable
from tqdm import tqdm

In [20]:
books = pd.read_csv("../data/books.csv")
train = pd.read_csv("../data/train_ratings.csv")
test = pd.read_csv("../data/test_ratings.csv")
users = pd.read_csv("../data/users.csv")
sub = pd.read_csv("../data/sample_submission.csv")

In [21]:
ids = pd.concat([train['user_id'], sub['user_id']]).unique()
isbns = pd.concat([train['isbn'], sub['isbn']]).unique()

idx2user = {idx:id for idx, id in enumerate(ids)}
idx2isbn = {idx:isbn for idx, isbn in enumerate(isbns)}

user2idx = {id:idx for idx, id in idx2user.items()}
isbn2idx = {isbn:idx for idx, isbn in idx2isbn.items()}

train['user_id'] = train['user_id'].map(user2idx)
sub['user_id'] = sub['user_id'].map(user2idx)

train['isbn'] = train['isbn'].map(isbn2idx)
sub['isbn'] = sub['isbn'].map(isbn2idx)


In [22]:
def image_vector(path):
    img = Image.open(path)
    scale = transforms.Resize((32, 32))
    tensor = transforms.ToTensor()
    img_fe = Variable(tensor(scale(img)))
    return img_fe

In [24]:
#img_train 생성
books_ = books.copy()
books_['isbn'] = books_['isbn'].map(isbn2idx)

df = train.copy()
train1 = True

if train1 == True:
    df_ = df.copy()
else:
    df_ = df.copy()
    df_['user_id'] = df_['user_id'].map(user2idx)
    df_['isbn'] = df_['isbn'].map(isbn2idx)

df_ = pd.merge(df_, books_[['isbn', 'img_path']], on='isbn', how='left')
df_['img_path'] = df_['img_path'].apply(lambda x: 'data/'+x)
img_vector_df = df_[['img_path']].drop_duplicates().reset_index(drop=True).copy()
data_box = []
print(img_vector_df['img_path'])
for idx, path in enumerate(sorted(img_vector_df['img_path'])):
    data = image_vector("../"+path)
    if data.size()[0] == 3:
        data_box.append(np.array(data))
    else:
        data_box.append(np.array(data.expand(3, data.size()[1], data.size()[2])))
img_vector_df['img_vector'] = data_box
df_ = pd.merge(df_, img_vector_df, on='img_path', how='left')

0         data/images/0002005018.01.THUMBZZZ.jpg
1         data/images/0060973129.01.THUMBZZZ.jpg
2         data/images/0374157065.01.THUMBZZZ.jpg
3         data/images/0399135782.01.THUMBZZZ.jpg
4         data/images/0425176428.01.THUMBZZZ.jpg
                           ...                  
129772    data/images/0743525493.01.THUMBZZZ.jpg
129773    data/images/067161746X.01.THUMBZZZ.jpg
129774    data/images/0884159221.01.THUMBZZZ.jpg
129775    data/images/0912333022.01.THUMBZZZ.jpg
129776    data/images/1569661057.01.THUMBZZZ.jpg
Name: img_path, Length: 129777, dtype: object


In [25]:
df_

,user_id,isbn,rating,img_path,img_vector
0,0,0,4,data/images/0002005018.01.THUMBZZZ.jpg,"[[[0.003921569, 0.003921569, 0.003921569, 0.00..."
1,1,0,7,data/images/0002005018.01.THUMBZZZ.jpg,"[[[0.003921569, 0.003921569, 0.003921569, 0.00..."
2,2,0,8,data/images/0002005018.01.THUMBZZZ.jpg,"[[[0.003921569, 0.003921569, 0.003921569, 0.00..."
3,3,0,8,data/images/0002005018.01.THUMBZZZ.jpg,"[[[0.003921569, 0.003921569, 0.003921569, 0.00..."
4,4,0,9,data/images/0002005018.01.THUMBZZZ.jpg,"[[[0.003921569, 0.003921569, 0.003921569, 0.00..."
...,...,...,...,...,...
306790,6313,129772,7,data/images/0743525493.01.THUMBZZZ.jpg,"[[[0.52156866, 0.53333336, 0.53333336, 0.49803..."
306791,1879,129773,6,data/images/067161746X.01.THUMBZZZ.jpg,"[[[0.79607844, 0.85882354, 0.80784315, 0.82745..."
306792,1879,129774,7,data/images/0884159221.01.THUMBZZZ.jpg,"[[[0.8980392, 0.9254902, 0.93333334, 0.9294117..."
306793,1879,129775,7,data/images/0912333022.01.THUMBZZZ.jpg,"[[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0..."


In [28]:
books_ = books.copy()
books_['isbn'] = books_['isbn'].map(isbn2idx)

df1 = test.copy()

# if train == True:
#     df_ = df.copy()
# else:
df_1 = df1.copy()
df_1['user_id'] = df_1['user_id'].map(user2idx)
df_1['isbn'] = df_1['isbn'].map(isbn2idx)

df_1= pd.merge(df_1, books_[['isbn', 'img_path']], on='isbn', how='left')
df_1['img_path'] = df_1['img_path'].apply(lambda x: 'data/'+x)
img_vector_df = df_1[['img_path']].drop_duplicates().reset_index(drop=True).copy()
data_box = []
for idx, path in tqdm(enumerate(sorted(img_vector_df['img_path']))):
    data = image_vector("../"+path)
    if data.size()[0] == 3:
        data_box.append(np.array(data))
    else:
        data_box.append(np.array(data.expand(3, data.size()[1], data.size()[2])))
img_vector_df['img_vector'] = data_box
df_1 = pd.merge(df_1, img_vector_df, on='img_path', how='left')

52000it [00:21, 2446.80it/s]


In [30]:
df_1

,user_id,isbn,rating,img_path,img_vector
0,13,0,0,data/images/0002005018.01.THUMBZZZ.jpg,"[[[0.99215686, 0.99607843, 0.9882353, 0.996078..."
1,13426,0,0,data/images/0002005018.01.THUMBZZZ.jpg,"[[[0.99215686, 0.99607843, 0.9882353, 0.996078..."
2,26761,1,0,data/images/0060973129.01.THUMBZZZ.jpg,"[[[0.40784314, 0.59607846, 0.6313726, 0.411764..."
3,16495,2,0,data/images/0374157065.01.THUMBZZZ.jpg,"[[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0..."
4,6225,3,0,data/images/0399135782.01.THUMBZZZ.jpg,"[[[0.003921569, 0.003921569, 0.003921569, 0.00..."
...,...,...,...,...,...
76694,7728,149565,0,data/images/1576734218.01.THUMBZZZ.jpg,"[[[0.003921569, 0.003921569, 0.003921569, 0.00..."
76695,47785,149566,0,data/images/3492223710.01.THUMBZZZ.jpg,"[[[0.18039216, 0.20784314, 0.12156863, 0.03529..."
76696,4209,149567,0,data/images/1896095186.01.THUMBZZZ.jpg,"[[[0.16078432, 0.11372549, 0.21176471, 0.41568..."
76697,40779,149568,0,data/images/8408044079.01.THUMBZZZ.jpg,"[[[0.1254902, 0.11372549, 0.105882354, 0.09803..."


In [29]:
img_train = df_
img_test = df_1

data = {
        'train':train,
        'test':test,
        'users':users,
        'books':books,
        'sub':sub,
        'idx2user':idx2user,
        'idx2isbn':idx2isbn,
        'user2idx':user2idx,
        'isbn2idx':isbn2idx,
        'img_train':img_train,
        'img_test':img_test,
        }

In [32]:
X_train, X_valid, y_train, y_valid = train_test_split(
                                                        data['img_train'][['user_id', 'isbn', 'img_vector']],
                                                        data['img_train']['rating'],
                                                        test_size=0.2,
                                                        random_state=42,
                                                        shuffle=True
                                                        )
data['X_train'], data['X_valid'], data['y_train'], data['y_valid'] = X_train, X_valid, y_train, y_valid

In [34]:
class Image_Dataset(Dataset):
    def __init__(self, user_isbn_vector, img_vector, label):
        self.user_isbn_vector = user_isbn_vector
        self.img_vector = img_vector
        self.label = label
    def __len__(self):
        return self.user_isbn_vector.shape[0]
    def __getitem__(self, i):
        return {
                'user_isbn_vector' : torch.tensor(self.user_isbn_vector[i], dtype=torch.long),
                'img_vector' : torch.tensor(self.img_vector[i], dtype=torch.float32),
                'label' : torch.tensor(self.label[i], dtype=torch.float32),
                }

In [35]:
BATCH_SIZE = 1024
train_dataset = Image_Dataset(
                                data['X_train'][['user_id', 'isbn']].values,
                                data['X_train']['img_vector'].values,
                                data['y_train'].values
                                )
valid_dataset = Image_Dataset(
                            data['X_valid'][['user_id', 'isbn']].values,
                            data['X_valid']['img_vector'].values,
                            data['y_valid'].values
                            )
test_dataset = Image_Dataset(
                            data['img_test'][['user_id', 'isbn']].values,
                            data['img_test']['img_vector'].values,
                            data['img_test']['rating'].values
                            )

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, num_workers=0, shuffle=True)
valid_dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, num_workers=0, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, num_workers=0, shuffle=False)
data['train_dataloader'], data['valid_dataloader'], data['test_dataloader'] = train_dataloader, valid_dataloader, test_dataloader


In [ ]:
class FeaturesEmbedding(nn.Module):

    def __init__(self, field_dims: np.ndarray, embed_dim: int):
        super().__init__()
        self.embedding = torch.nn.Embedding(sum(field_dims), embed_dim)
        self.offsets = np.array((0, *np.cumsum(field_dims)[:-1]), dtype=np.long)
        torch.nn.init.xavier_uniform_(self.embedding.weight.data)

    def forward(self, x: torch.Tensor):
        """
        :param x: Long tensor of size ``(batch_size, num_fields)``
        """
        x = x + x.new_tensor(self.offsets).unsqueeze(0)
        return self.embedding(x)

In [ ]:
class CNN_Base(nn.Module):
    def __init__(self, ):
        super(CNN_Base, self).__init__()
        self.cnn_layer = nn.Sequential(
                                        nn.Conv2d(3, 6, kernel_size=3, stride=2, padding=1),
                                        nn.BatchNorm2d(6),
                                        nn.ReLU(),
                                        nn.MaxPool2d(kernel_size=3, stride=2),
                                        nn.Conv2d(6, 12, kernel_size=3, stride=2, padding=1),
                                        nn.BatchNorm2d(12),
                                        nn.ReLU(),
                                        )
    def forward(self, x):
        x = self.cnn_layer(x)
        x = x.view(-1, 12 * 4 * 4)
        return x


In [ ]:
class FactorizationMachine_v(nn.Module):

    def __init__(self, input_dim, latent_dim):
        super().__init__()
        self.v = nn.Parameter(torch.rand(input_dim, latent_dim), requires_grad = True)
        self.linear = nn.Linear(input_dim, 1, bias=True)

    def forward(self, x):
        linear = self.linear(x)
        square_of_sum = torch.mm(x, self.v) ** 2
        sum_of_square = torch.mm(x ** 2, self.v ** 2)
        pair_interactions = torch.sum(square_of_sum - sum_of_square, dim=1, keepdim=True)
        output = linear + (0.5 * pair_interactions)
        return output

In [ ]:
class _CNN_FM(torch.nn.Module):
    def __init__(self, field_dims, embed_dim, latent_dim):
        super().__init__()
        self.embedding = FeaturesEmbedding(field_dims, embed_dim)
        self.cnn = CNN_Base()
        self.fm = FactorizationMachine_v(
                                         input_dim=(embed_dim * 2) + (12 * 4 * 4),
                                         latent_dim=latent_dim,
                                         )

    def forward(self, x):
        user_isbn_vector, img_vector = x[0], x[1]
        user_isbn_feature = self.embedding(user_isbn_vector)
        img_feature = self.cnn(img_vector)
        feature_vector = torch.cat([
                                    user_isbn_feature.view(-1, user_isbn_feature.size(1) * user_isbn_feature.size(2)),
                                    img_feature
                                    ], dim=1)
        output = self.fm(feature_vector)
        return output.squeeze(1)

In [36]:
class CNN_FM:
    def __init__(self, args, data):
        super().__init__()
        self.device = args.DEVICE
        self.model = _CNN_FM(
                            np.array([len(data['user2idx']), len(data['isbn2idx'])], dtype=np.uint32),
                            args.CNN_FM_EMBED_DIM,
                            args.CNN_FM_LATENT_DIM
                            ).to(self.device)
        self.optimizer =  torch.optim.Adam(self.model.parameters(), lr=args.LR)
        self.train_data_loader = data['train_dataloader']
        self.valid_data_loader = data['valid_dataloader']
        self.criterion = RMSELoss()
        self.epochs = args.EPOCHS
        self.model_name = 'image_model'


    def train(self):
        minimum_loss = 999999999
        loss_list = []
        tk0 = tqdm.tqdm(range(self.epochs), smoothing=0, mininterval=1.0)
        for epoch in tk0:
            self.model.train()
            total_loss = 0
            n = 0
            for i, data in enumerate(self.train_data_loader):
                if len(data) == 2:
                    fields, target = [data['user_isbn_vector'].to(self.device)], data['label'].to(self.device)
                else:
                    fields, target = [data['user_isbn_vector'].to(self.device), data['img_vector'].to(self.device)], data['label'].to(self.device)
                y = self.model(fields)
                loss = self.criterion(y, target.float())
                self.model.zero_grad()
                loss.backward()
                self.optimizer.step()
                total_loss += loss.item()
                n += 1
            self.model.eval()
            val_total_loss = 0
            val_n = 0
            for i, data in enumerate(self.valid_data_loader):
                if len(data) == 2:
                    fields, target = [data['user_isbn_vector'].to(self.device)], data['label'].to(self.device)
                else:
                    fields, target = [data['user_isbn_vector'].to(self.device), data['img_vector'].to(self.device)], data['label'].to(self.device)
                y = self.model(fields)
                loss = self.criterion(y, target.float())
                self.model.zero_grad()
                loss.backward()
                self.optimizer.step()
                val_total_loss += loss.item()
                val_n += 1
            if minimum_loss > (val_total_loss/val_n):
                minimum_loss = (val_total_loss/val_n)
                if not os.path.exists('./models'):
                    os.makedirs('./models')
                torch.save(self.model.state_dict(), './models/{}.pt'.format(self.model_name))
                loss_list.append([epoch, total_loss/n, val_total_loss/val_n, 'Model saved'])
            else:
                loss_list.append([epoch, total_loss/n, val_total_loss/val_n, 'None'])
            tk0.set_postfix(train_loss=total_loss/n, valid_loss=val_total_loss/val_n)
            wandb.log({"total_loss": total_loss/n}, step = epoch)
            wandb.log({"val_loss": val_total_loss/n}, step = epoch)
        return val_total_loss/n

    def predict(self, test_data_loader):
        self.model.eval()
        self.model.load_state_dict(torch.load('./models/{}.pt'.format(self.model_name)))
        targets, predicts = list(), list()
        with torch.no_grad():
            for data in test_data_loader:
                if len(data) == 2:
                    fields, target = [data['user_isbn_vector'].to(self.device)], data['label'].to(self.device)
                else:
                    fields, target = [data['user_isbn_vector'].to(self.device), data['img_vector'].to(self.device)], data['label'].to(self.device)
                y = self.model(fields)
                targets.extend(target.tolist())
                predicts.extend(y.tolist())
        return predicts